In [2]:
import os
import yaml
from dotenv import load_dotenv
from langchain_google_genai import GoogleGenerativeAIEmbeddings, GoogleGenerativeAI

In [3]:
env_path = r"D:\common_credentials\.env"
load_dotenv(dotenv_path=env_path)

True

In [7]:
from langchain_groq import ChatGroq
llm= ChatGroq(model="llama-3.2-11b-vision-preview")
print(llm.invoke("HI"))

content='How can I assist you today?' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 36, 'total_tokens': 44, 'completion_time': 0.061578703, 'prompt_time': 0.023836374, 'queue_time': 0.017403141999999996, 'total_time': 0.085415077}, 'model_name': 'llama-3.2-11b-vision-preview', 'system_fingerprint': 'fp_9cb648b966', 'finish_reason': 'stop', 'logprobs': None} id='run-0adf574e-6abd-4ea8-8b65-2d26a2cf85a1-0' usage_metadata={'input_tokens': 36, 'output_tokens': 8, 'total_tokens': 44}


# RAG

In [11]:
from langchain_community.document_loaders import WebBaseLoader
loader= WebBaseLoader(web_path="https://python.langchain.com/docs/introduction/")
data= loader.load()

In [20]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter= RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=150)
docs= splitter.split_documents(data)
len(docs)

37

In [29]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embedding_model= GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
len(embedding_model.embed_query("Hi"))

768

In [32]:
from langchain.vectorstores import FAISS, Pinecone
vectorstore= FAISS.from_documents(docs, embedding_model)
vectorstore

In [35]:
query= "what is langchain"
result= vectorstore.similarity_search_with_score(query, k=2)
result

[(Document(id='6b8117fb-184b-4a77-b21e-0568f76031a6', metadata={'source': 'https://python.langchain.com/docs/introduction/', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='LangChain is a framework for developing applications powered by large language models (LLMs).\nLangChain simplifies every stage of the LLM application lifecycle:'),
  0.44526535),
 (Document(id='f5f3dc52-d8ee-43f6-98ff-33b8c163194d', metadata={'source': 'https://python.langchain.com/docs/introduction/', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content="Introductions to all the key parts of LangChain you’ll need to know! Here you'll find high level explanations of all LangChain concepts.\nFor a deeper dive into LangGraph concepts, check out this pa

In [69]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser

output_parser=JsonOutputParser()
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

#basic chain 

chain1= prompt | llm | output_parser
print("chain:", chain1)
print("-------------------------------\n")
print(chain1.invoke({'query':'what is langchain'}))


chain: first=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={'format_instructions': 'Return a JSON object.'}, template='Answer the user query.\n{format_instructions}\n{query}\n') middle=[ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002335FDE3990>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000023362B7D0D0>, model_name='llama-3.2-11b-vision-preview', model_kwargs={}, groq_api_key=SecretStr('**********'))] last=JsonOutputParser()
-------------------------------

{'name': 'LangChain', 'description': 'A Python framework for building and managing chains of AI models, data sources, and other tools.', 'tags': ['AI', 'Machine Learning', 'Natural Language Processing'], 'features': ['Support for various AI models, including LLMs and other models', 'Ability to create chains of models and data sources', 'Support for data storage and retrieval', 'Integration with various APIs and tools'], 'use_cases': [{'ti

In [70]:
chain1

PromptTemplate(input_variables=['query'], input_types={}, partial_variables={'format_instructions': 'Return a JSON object.'}, template='Answer the user query.\n{format_instructions}\n{query}\n')
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002335FDE3990>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000023362B7D0D0>, model_name='llama-3.2-11b-vision-preview', model_kwargs={}, groq_api_key=SecretStr('**********'))
| JsonOutputParser()

In [49]:
from langchain.prompts import ChatPromptTemplate
#this prompt is for generating the ans based on retrieved context
generation_prompt= ChatPromptTemplate.from_template(""" Answer the following question based only on provided
context:
<context>
{context}
</context>""")
prompt

ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template=' Answer the following question based only on provided\ncontext:\n<context>\n{context}\n</context>'), additional_kwargs={})])

In [48]:
generation_prompt.invoke({"context":"It's very hot today"}) # sample

ChatPromptValue(messages=[HumanMessage(content=" Answer the following question based only on provided\ncontext:\n<context>\nIt's very hot today\n</context>", additional_kwargs={}, response_metadata={})])

In [68]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain= create_stuff_documents_chain(llm, generation_prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template=' Answer the following question based only on provided\ncontext:\n<context>\n{context}\n</context>'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002335FDE3990>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000023362B7D0D0>, model_name='llama-3.2-11b-vision-preview', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

## creating chains

In [75]:
retriever= vectorstore.as_retriever()
# retriever.get_relevant_documents("what is lanchain")

In [76]:
from langchain.prompts import ChatPromptTemplate
#this prompt is for generating the ans based on retrieved context
generation_prompt= ChatPromptTemplate.from_template(""" Answer the following question based only on provided
context:
<context>
{context}
</context>""")
prompt

PromptTemplate(input_variables=['query'], input_types={}, partial_variables={'format_instructions': 'Return a JSON object.'}, template='Answer the user query.\n{format_instructions}\n{query}\n')

In [80]:
from langchain.chains.combine_documents import create_stuff_documents_chain
## this chain will combine the documents get from retriever and pass to the llm through template for final generatin
generation_chain= create_stuff_documents_chain(llm, generation_prompt)
generation_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template=' Answer the following question based only on provided\ncontext:\n<context>\n{context}\n</context>'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002335FDE3990>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000023362B7D0D0>, model_name='llama-3.2-11b-vision-preview', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [ ]:
# Retreval Chain=This chain takes in a user inquiry, which is then passed to the retriever to fetch relevant documents. Those documents (and original inputs) are then passed to an LLM to generate a response

In [84]:
from langchain.chains import create_retrieval_chain
# this chain will combine retriever & generation prompt (chatprompt | llm)
retrival_chain= create_retrieval_chain(retriever, generation_chain)
retrival_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000233798F2510>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template=' Answer the following question based only on provided\ncontext:\n<context>\n{context}\n</context>'), additional_kwargs={})])
            

In [91]:
%%time
result= retrival_chain.invoke({"input":"What is langchain"})
print(result)

{'input': 'What is langchain', 'context': [Document(id='6b8117fb-184b-4a77-b21e-0568f76031a6', metadata={'source': 'https://python.langchain.com/docs/introduction/', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='LangChain is a framework for developing applications powered by large language models (LLMs).\nLangChain simplifies every stage of the LLM application lifecycle:'), Document(id='f5f3dc52-d8ee-43f6-98ff-33b8c163194d', metadata={'source': 'https://python.langchain.com/docs/introduction/', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content="Introductions to all the key parts of LangChain you’ll need to know! Here you'll find high level explanations of all LangChain concepts.\nFor a deeper dive into LangGraph con

In [97]:
%%time
result2= retrival_chain.invoke({"input":"What is langgraph?"})
print(result2)

{'input': 'What is langgraph?', 'context': [Document(id='19938b6d-3927-4f90-957b-2f7a4cfc7b66', metadata={'source': 'https://python.langchain.com/docs/introduction/', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='Trace and evaluate your language model applications and intelligent agents to help you move from prototype to production.\n🦜🕸️ LangGraph\u200b\nBuild stateful, multi-actor applications with LLMs. Integrates smoothly with LangChain, but can be used without it. LangGraph powers production-grade agents, trusted by Linkedin, Uber, Klarna, GitLab, and many more.\nAdditional resources\u200b\nVersions\u200b\nSee what changed in v0.3, learn how to migrate legacy code, read up on our versioning policies, and more.'), Document(id='e48e0e10-1216-4936-97ff-c95d28a56728', metadata={'source': 'https://python.langchain.com/docs/introduction/', 'title': '

In [98]:
result2['answer']

"Based on the provided context, I will answer the following questions:\n\n1. What is LangGraph used for?\n\nLangGraph is used to build stateful, multi-actor applications with Large Language Models (LLMs). It integrates smoothly with LangChain, but can be used without it, and it powers production-grade agents.\n\n2. What are some notable companies that trust LangGraph?\n\nNotable companies that trust LangGraph include LinkedIn, Uber, Klarna, and GitLab.\n\n3. What is the purpose of LangSmith?\n\nLangSmith is used to inspect, monitor, and evaluate applications, so that they can be continuously optimized and deployed with confidence.\n\n4. What is the LangChain ecosystem?\n\nThe LangChain ecosystem is a rich collection of tools that integrate with LangChain's framework and build on top of it."

# Summary

In [ ]:
#Rerieval chain: takes the user query retrieve relevant context & pass to the generation cchain1
#create stuff doc chain: combines the llm with generation prompt, receives the docs from retriever